In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from mesh_import import mesh
import mesh.affix as affix
from mesh.affix import CkipAffixoids
from mesh.affix import AffixoidCtmDataset, ByCharCtmProcessor

<ipython-input-2-b362af8f5086>:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Testing By Char Processor

In [5]:
proc = affix.ByCharCtmProcessor(debug=True, no_cache=True)

Debug mode is ON


loading ASBC corpus streamer index


no_cache is TRUE, all cache is ignored


In [9]:
list(proc.example_store.keys())

['時_', '_間', '三_', '_月', '十_', '_日', '星_', '_四', '上_', '_午', '_時']

In [10]:
proc.charloc_sentences

{'時_': [('時間',
   [('醫師', 'Na'),
    ('工作', 'VA'),
    ('時間', 'Na'),
    ('過', 'Dfa'),
    ('長', 'VH'),
    ('，', 'COMMACATEGORY')]),
  ('時間',
   [('「', 'PARENTHESISCATEGORY'),
    ('鼠', 'Na'),
    ('是', 'SHI'),
    ('時間', 'Na'),
    ('，', 'COMMACATEGORY')]),
  ('時間',
   [('也', 'D'),
    ('沒有', 'VJ'),
    ('什麼', 'Nep'),
    ('方法', 'Na'),
    ('可以', 'D'),
    ('去除', 'VC'),
    ('時間', 'Na'),
    ('或', 'Caa'),
    ('貶抑', 'VC'),
    ('時間', 'Na'),
    ('。', 'PERIODCATEGORY')]),
  ('時間',
   [('在', 'P'),
    ('總預算', 'Na'),
    ('增長', 'VHC'),
    ('有限', 'VH'),
    ('的', 'DE'),
    ('這', 'Nep'),
    ('段', 'Nf'),
    ('時間', 'Na'),
    ('裡', 'Ng'),
    ('，', 'COMMACATEGORY')]),
  ('時間',
   [('在', 'P'),
    ('極', 'Dfa'),
    ('短', 'VH'),
    ('地', 'DE'),
    ('時間', 'Na'),
    ('內', 'Ng'),
    ('就', 'D'),
    ('已', 'D'),
    ('遍佈', 'VH'),
    ('世界', 'Nc'),
    ('各', 'Nes'),
    ('地', 'Na'),
    ('的', 'DE'),
    ('學術', 'Na'),
    ('單位', 'Na'),
    ('。', 'PERIODCATEGORY')]),
  ('時間',
   [('搶', 'VD'),